# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform
import SimpleITK as sitk


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


XGBoost Version: 2.1.4


In [4]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

In [41]:
moving_name = 'MD589'
fixed_name = 'Allen'
moving_all = fetch_coms(moving_name, scaling_factor=1)
fixed_all = list_coms(fixed_name, scaling_factor=1)
common_keys = list(moving_all.keys() & fixed_all.keys())
moving_src = np.array([moving_all[s] for s in common_keys])
fixed_src = np.array([fixed_all[s] for s in common_keys])
print(len(common_keys))

37


In [42]:
#transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
#                                                  scaling_factor=1)
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(transformation_matrix)

[[ 9.96008343e-01  1.69780452e-01  2.72223202e-02 -2.95710421e+03]
 [-8.25783293e-02  1.18030608e+00  1.09038636e-01 -2.89953155e+03]
 [ 1.88559100e-03 -7.08043196e-02  1.09790465e+00  1.21989539e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [43]:
# these keys have very high errors: 'RtTg', 'AP'
midbrain_keys = {
            "3N_L",
            "3N_R",
            "4N_L",
            "4N_R",
            "IC",
            "PBG_L",
            "PBG_R",
            "SC",
            "SNC_L",
            "SNC_R",
            "SNR_L",
            "SNR_R",
        }
bad_keys = ('RtTg', 'AP', '3N_L', '3N_R')
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
good_keys = set(common_keys) - set(bad_keys)
print(f'#good_keys={len(good_keys)}')

#good_keys=35


In [44]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = apply_affine_transform(moving0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 disk to Allen RMS 237.0680595008574
# MD585 to MD589 RMS 18.2658167690059

RMS 266.1973052186457


In [46]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#com = [10949, 3923, 4399]
print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
transformed_structure = apply_affine_transform(com, transformation_matrix)
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
print(f'{moving_name}-{fixed_name} {structure} {np.round(diff)}')


MD589 SC non trans [11339.  4729.  4288.]
MD589 SC apply trans [9256. 2214. 5614.]
Allen SC [9140. 2388. 5692.]
MD589-Allen SC [ 115. -174.  -78.]


In [39]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)

,structure,MD585,Allen,transformed,difference,sumsquares
Index,,,,,,
3,VLL_L,"[1046.0, 574.0, 283.0]","[946.0, 518.0, 389.0]","[940.0, 521.0, 393.0]","[-7.0, 3.0, 4.0]",8.8153
11,5N_R,"[1140.0, 549.0, 565.0]","[1019.0, 529.0, 729.0]","[1018.0, 521.0, 726.0]","[-1.0, -8.0, -3.0]",8.8220
34,LRt_L,"[1355.0, 696.0, 288.0]","[1234.0, 699.0, 439.0]","[1229.0, 701.0, 430.0]","[-6.0, 2.0, -10.0]",11.3613
36,6N_L,"[1197.0, 558.0, 381.0]","[1077.0, 521.0, 529.0]","[1078.0, 525.0, 517.0]","[1.0, 3.0, -12.0]",12.1098
14,SNR_R,"[967.0, 557.0, 582.0]","[844.0, 517.0, 735.0]","[856.0, 514.0, 733.0]","[12.0, -3.0, -2.0]",12.2102
13,6N_R,"[1198.0, 557.0, 451.0]","[1077.0, 521.0, 609.0]","[1076.0, 528.0, 599.0]","[-1.0, 6.0, -10.0]",12.2363
9,3N_L,"[1032.0, 457.0, 425.0]","[910.0, 379.0, 552.0]","[922.0, 388.0, 550.0]","[11.0, 9.0, -2.0]",14.4258
1,IC,"[1144.0, 312.0, 439.0]","[1040.0, 232.0, 568.0]","[1027.0, 226.0, 566.0]","[-13.0, -6.0, -2.0]",14.8451
18,4N_L,"[1065.0, 458.0, 413.0]","[959.0, 378.0, 543.0]","[953.0, 392.0, 539.0]","[-6.0, 14.0, -4.0]",15.8030


In [ ]:
com_um = np.array([11339,  4729,  4288.])
origin_allen = np.array([1013, 406, 254])
test_com = np.array([121,  67, 175])
scale = np.array([0.452, 0.452, 20])
origin_neuro = origin_um / scale
volume_shape = np.array([158, 102, 182])
xy_resolution = 0.452
SCALING_FACTOR = 32
scale0 = np.array([xy_resolution*SCALING_FACTOR, xy_resolution*SCALING_FACTOR, 20])
allen_um = np.array([10,10,10])
scale_allen = scale0 / allen_um

In [ ]:
volume_shape = np.array([158, 102, 182])
xy_resolution = 0.452
SCALING_FACTOR = 20
to_um = np.array([0.452*32, 0.452*32, 20])
(volume_shape * to_um) / SCALING_FACTOR